In [1]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import pickle

Welcome to ROOTaaS 6.06/08


In [30]:
batch_size_jets = 25
batch_size_tracks = 25
read_pos_jets = 0
read_pos_tracks = 0

In [31]:
d1 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree", start = read_pos_jets, stop = read_pos_jets + batch_size_jets))

In [32]:
d2 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree_track", start = read_pos_tracks, stop = read_pos_tracks + batch_size_tracks))

In [33]:
last_tracks = (int)(d2.tail(1)['Track_jetIndex'].iloc[0]-1)

In [34]:
last_jet = (int)(d1.tail(1)['Jet_jetIndex'].iloc[0]-1)

In [35]:
read_pos_jets += (d1.loc[d1['Jet_jetIndex'] == last_tracks].index[-1] + 1)
read_pos_tracks += (d2.loc[d2['Track_jetIndex'] == last_tracks].index[-1] + 1)

In [36]:
d1['track_data'] = pd.np.empty((len(d1.index),0)).tolist()

In [37]:
# iterate over the track list to join jets with the tracks belonging to them
for irow, row in d2.iterrows():
    # these are the track data of the current track:
    tracks = row[["Track_pt", "Track_eta", "Track_phi", "Track_dxy", "Track_dz", "Track_IP", "Track_IP2D", "Track_length"]].as_matrix()
    jet_index = int(row["Track_jetIndex"])
    if jet_index > last_tracks:
        break
    table_index = d1.loc[d1['Jet_jetIndex'] == jet_index].index[0]
    
    # append the tracks data to the matching jet in the main table
    d1['track_data'][table_index].append(tracks)

In [38]:
# now divide the jets and put them in separate lists, according to their flavour
jets_b = []
jets_l = []
jets_c = []

# iterate over the jet list, with already matched tracks
for irow, row in d1.iterrows():
    jet_index = int(row["Jet_jetIndex"])
    if jet_index > last_tracks:
        break
    
    flavour = int(row["Jet_flavour"])
    
    # select the right list this jet belongs to
    if abs(flavour) == 5:
        jets = jets_b
    elif abs(flavour) == 4:
        jets = jets_c
    else:
        jets = jets_l
        
    # add the new jet to the list
    jets += [(row["Jet_pt"], row["Jet_eta"], row["Jet_phi"], row["Jet_mass"], flavour, row["track_data"])]

In [44]:
jets_b[0][-1][0][0]

2.5390625